In [2]:
import os
import numpy as np
import cv2 as cv
from glob import glob
from tqdm import tqdm #progressbar
from tifffile import imread
import tifffile
import albumentations as A
import imageio
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from skimage import data
from skimage import filters
import logging
import PIL
import scipy.ndimage 
import time
import math
import random
from sklearn.metrics import confusion_matrix
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import pandas as pd
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from tensorflow.keras import backend as K
from sklearn.utils import shuffle
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import dabest

Define U-Net Architecture


In [3]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

if __name__ == "__main__":
    input_shape = (512, 512, 3)
    model = build_unet(input_shape)
    model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

                                                                                                  
 conv2d_8 (Conv2D)              (None, 32, 32, 1024  4719616     ['max_pooling2d_3[0][0]']        
                                )                                                                 
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 32, 32, 1024  4096       ['conv2d_8[0][0]']               
 rmalization)                   )                                                                 
                                                                                                  
 activation_8 (Activation)      (None, 32, 32, 1024  0           ['batch_normalization_8[0][0]']  
                                )                                                                 
                                                                                                  
 conv2d_9 

                                                                                                  
 activation_15 (Activation)     (None, 256, 256, 12  0           ['batch_normalization_15[0][0]'] 
                                8)                                                                
                                                                                                  
 conv2d_transpose_3 (Conv2DTran  (None, 512, 512, 64  32832      ['activation_15[0][0]']          
 spose)                         )                                                                 
                                                                                                  
 concatenate_3 (Concatenate)    (None, 512, 512, 12  0           ['conv2d_transpose_3[0][0]',     
                                8)                                'activation_1[0][0]']           
                                                                                                  
 conv2d_16

In [4]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

Data Augmentation 

In [5]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    train_x = sorted(glob(os.path.join(path,"training", "training_source", "*.tif")))
    train_y = sorted(glob(os.path.join(path,"training", "training_target", "*.tif")))

    test_x = sorted(glob(os.path.join(path,"valid", "image", "*.tif")))
    test_y = sorted(glob(os.path.join(path,"valid", "mask", "*.tif")))
    
    return(train_x, train_y), (test_x, test_y)

def augment_data(images, masks, save_path, augment=True):
    size = (512,512)
    
    for idx, (x,y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        name = x.split("/")[-1].split(".")[0]
        name = x.split("\\")[-1].split(".")[0]
        
        x = cv.imread(x, cv.IMREAD_COLOR)
        y = imageio.mimread(y)[0]
        
        if augment == True:
            aug = A.HorizontalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1= augmented["mask"]
            
            aug = A.VerticalFlip(p=1.0)
            augmented = aug(image=x, mask=y)
            x2 = augmented["image"]
            y2= augmented["mask"]
            
            aug = A.ElasticTransform(p=1.0, alpha=120, sigma=120*0.5, alpha_affine=120)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3= augmented["mask"]
        
            X= [x, x1, x2, x3]
            Y= [y, y1, y2, y3]
            
            
        else:
            X = [x]
            Y = [y]
        
        index=0
                
        for i,m in zip(X,Y):
            i=cv.resize(i, size)
            m=cv.resize(m, size)
            
            tmp_image_name = f"{name}_{index}.tif"
            tmp_mask_name = f"{name}_{index}.tif"
            
            image_path = os.path.join(save_path, "image", tmp_image_name)
            mask_path = os.path.join(save_path, "mask", tmp_mask_name)
            
            cv.imwrite(image_path, i)
            cv.imwrite(mask_path, m)
            
            index+=1

            
    #Create directories to save augmented data
create_dir("C:/Users/17605/Code/gunpowder/Scratch_10x/new_data/train/image/")
create_dir("C:/Users/17605/Code/gunpowder/Scratch_10x/new_data/train/mask/")
create_dir("C:/Users/17605/Code/gunpowder/Scratch_10x/new_data/valid/image/")
create_dir("C:/Users/17605/Code/gunpowder/Scratch_10x/new_data/valid/mask/")
    
if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    
    data_path = "C:/Users/17605/Code/gunpowder/Scratch_10x/"
    (train_x, train_y), (test_x, test_y) = load_data(data_path)
    print(f"Train: {len(train_x)} -  {len(train_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")
    
    #Data augmentation
    augment_data(train_x, train_y, "C:/Users/17605/Code/gunpowder/Scratch_10x/new_data/train/", augment=True)
    augment_data(test_x, test_y, "C:/Users/17605/Code/gunpowder/Scratch_10x/new_data/valid/", augment=False)

Train: 2318 -  2318
Test: 101 - 101


  1%|▉                                                                               | 27/2318 [00:14<20:00,  1.91it/s]


KeyboardInterrupt: 

Initial Training of Unet model:

In [16]:
#train
H = 512
W = 512

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.tif")))
    y = sorted(glob(os.path.join(path, "mask", "*.tif")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv.imread(path, cv.IMREAD_COLOR)
    # x = cv.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv.imread(path, cv.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory to save files """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 2
    lr = 1e-4
    num_epochs = 100
    model_path = os.path.join("files", "model.h5")
    csv_path = os.path.join("files", "data.csv")

    """ Dataset """
    dataset_path = "C:/Users/17605/Code/gunpowder/Scratch_10x/new_data"
    train_path = os.path.join(dataset_path, "train")
    valid_path = os.path.join(dataset_path, "valid")

    train_x, train_y = load_data(train_path)
    train_x, train_y = shuffling(train_x, train_y)
    valid_x, valid_y = load_data(valid_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1
    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = build_unet((H, W, 3))
    model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train: 9616 - 9616
Valid: 101 - 101
Epoch 1/100
4808/4808 [==============================] - 1341s 278ms/step - loss: 0.0234 - dice_coef: 0.9766 - iou: 0.9554 - recall_7: 0.9750 - precision_7: 0.9842 - val_loss: 0.0109 - val_dice_coef: 0.9891 - val_iou: 0.9785 - val_recall_7: 0.9760 - val_precision_7: 0.9987

Epoch 00001: val_loss improved from inf to 0.01089, saving model to files\model.h5
Epoch 2/100
4808/4808 [==============================] - 1327s 276ms/step - loss: 0.0149 - dice_coef: 0.9851 - iou: 0.9710 - recall_7: 0.9814 - precision_7: 0.9893 - val_loss: 0.0111 - val_dice_coef: 0.9889 - val_iou: 0.9781 - val_recall_7: 0.9750 - val_precision_7: 0.9988

Epoch 00002: val_loss did not improve from 0.01089
Epoch 3/100
4808/4808 [==============================] - 1332s 277ms/step - loss: 0.0137 - dice_coef: 0.9863 - iou: 0.9732 - recall_7: 0.9824 - precision_7: 0.9904 - val_loss: 0.0337 - val_dice_coef: 0.9661 - val_iou: 0.9346 - val_recall_7: 0.9269 - val_precision_7: 0.9999

Epoch


Epoch 00024: val_loss improved from 0.00733 to 0.00728, saving model to files\model.h5
Epoch 25/100
4808/4808 [==============================] - 1331s 277ms/step - loss: 0.0087 - dice_coef: 0.9913 - iou: 0.9829 - recall_7: 0.9874 - precision_7: 0.9949 - val_loss: 0.0072 - val_dice_coef: 0.9928 - val_iou: 0.9858 - val_recall_7: 0.9865 - val_precision_7: 0.9977

Epoch 00025: val_loss improved from 0.00728 to 0.00719, saving model to files\model.h5
Epoch 26/100
4808/4808 [==============================] - 1331s 277ms/step - loss: 0.0086 - dice_coef: 0.9914 - iou: 0.9830 - recall_7: 0.9874 - precision_7: 0.9950 - val_loss: 0.0072 - val_dice_coef: 0.9928 - val_iou: 0.9856 - val_recall_7: 0.9858 - val_precision_7: 0.9979

Epoch 00026: val_loss did not improve from 0.00719
Epoch 27/100
4808/4808 [==============================] - 1330s 277ms/step - loss: 0.0085 - dice_coef: 0.9915 - iou: 0.9832 - recall_7: 0.9875 - precision_7: 0.9951 - val_loss: 0.0071 - val_dice_coef: 0.9929 - val_iou: 0.9


Epoch 00048: val_loss did not improve from 0.00660
Epoch 49/100
4808/4808 [==============================] - 1334s 277ms/step - loss: 0.0072 - dice_coef: 0.9928 - iou: 0.9858 - recall_7: 0.9889 - precision_7: 0.9961 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_iou: 0.9869 - val_recall_7: 0.9869 - val_precision_7: 0.9981

Epoch 00049: val_loss did not improve from 0.00660
Epoch 50/100
4808/4808 [==============================] - 1350s 281ms/step - loss: 0.0071 - dice_coef: 0.9929 - iou: 0.9859 - recall_7: 0.9889 - precision_7: 0.9961 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_iou: 0.9869 - val_recall_7: 0.9869 - val_precision_7: 0.9981

Epoch 00050: val_loss improved from 0.00660 to 0.00660, saving model to files\model.h5
Epoch 51/100
4808/4808 [==============================] - 1390s 289ms/step - loss: 0.0071 - dice_coef: 0.9929 - iou: 0.9859 - recall_7: 0.9890 - precision_7: 0.9961 - val_loss: 0.0066 - val_dice_coef: 0.9934 - val_iou: 0.9869 - val_recall_7: 0.9868 - val_pre


Epoch 00072: val_loss improved from 0.00654 to 0.00652, saving model to files\model.h5
Epoch 73/100
4808/4808 [==============================] - 1328s 276ms/step - loss: 0.0069 - dice_coef: 0.9931 - iou: 0.9863 - recall_7: 0.9892 - precision_7: 0.9963 - val_loss: 0.0065 - val_dice_coef: 0.9935 - val_iou: 0.9870 - val_recall_7: 0.9869 - val_precision_7: 0.9981

Epoch 00073: val_loss did not improve from 0.00652
Epoch 74/100
4808/4808 [==============================] - 1328s 276ms/step - loss: 0.0069 - dice_coef: 0.9931 - iou: 0.9863 - recall_7: 0.9892 - precision_7: 0.9963 - val_loss: 0.0065 - val_dice_coef: 0.9935 - val_iou: 0.9870 - val_recall_7: 0.9869 - val_precision_7: 0.9981

Epoch 00074: val_loss did not improve from 0.00652
Epoch 75/100
4808/4808 [==============================] - 1328s 276ms/step - loss: 0.0069 - dice_coef: 0.9931 - iou: 0.9864 - recall_7: 0.9892 - precision_7: 0.9963 - val_loss: 0.0065 - val_dice_coef: 0.9935 - val_iou: 0.9870 - val_recall_7: 0.9869 - val_pre

Model Validation

In [51]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv.imread(path, cv.IMREAD_COLOR)
    # x = cv.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv.imread(path, cv.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.tif")))
    y = sorted(glob(os.path.join(path, "mask", "*.tif")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1)

    y_pred = np.expand_dims(y_pred, axis=-1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) * 255

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred], axis=1)
    cv.imwrite(save_image_path, cat_images)

if __name__ == "__main__":
    """ Save the results in this folder """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
        model = tf.keras.models.load_model("files/model.h5")

    """ Load the dataset """
    dataset_path = os.path.join("new_data", "test")
    test_x, test_y = load_data(dataset_path)

    """ Make the prediction and calculate the metrics values """
    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extracting name """
        name = x.split("\\")[-1].split(".")[0]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Prediction """
        y_pred = model.predict(np.expand_dims(x, axis=0))[0]
        y_pred = y_pred > 0.5
        y_pred = y_pred.astype(np.int32)
        y_pred = np.squeeze(y_pred, axis=-1)

        """ Saving the images """
        save_image_path = f"results/{name}.png"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculate the metrics """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(score)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    """ Saving """
    df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/2021_11_04_unet_score.csv")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:46<00:00,  2.19it/s]

[0.99409186 0.99319621 0.98648818 0.99545764 0.99094742]
Accuracy: 0.99409
F1: 0.99320
Jaccard: 0.98649
Recall: 0.99546
Precision: 0.99095


In [10]:
dataset_path= os.path.join('DIC/*')

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv.imread(path, cv.IMREAD_COLOR)
#     x = cv.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "*.tif")))
    return x

for subdir in glob(dataset_path):
    if __name__ == "__main__": 

        """ Load the model """
        with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
             model = tf.keras.models.load_model("files/model.h5")

        """ Load the dataset """
        test_x = sorted(glob(os.path.join(subdir, "*.tif")))
        folder_name= subdir.split("\\")[-1].split("-")[0]+"_Binary"
        create_dir(folder_name)
        """ Make the prediction and calculate the metrics values """
        i=0
        for x in tqdm(zip(test_x), total=len(test_x)):
            if i<len(test_x):
                """ Extracting name """
                name = x[i].split("\\")[-1].split(".")[0]
                
                """ Read the image """
                ori_x, x = read_image(x[i])
                
                """ Prediction """
                y_pred = model.predict(np.expand_dims(x, axis=0))[0]
                y_pred = y_pred > 0.5
                y_pred = y_pred.astype(np.int32)
                y_pred = np.squeeze(y_pred, axis=-1)

                """ Saving the images """
                save_image_path = f"{folder_name}/{name}.png"
                cv.imwrite(save_image_path, y_pred)
                i==i+1


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]

1/1 [==============================] - 0s 423ms/step


  0%|▍                                                                                 | 1/200 [00:00<01:39,  1.99it/s]

1/1 [==============================] - 0s 44ms/step


  1%|▊                                                                                 | 2/200 [00:00<00:58,  3.39it/s]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 198/200 [00:23<00:00,  9.18it/s]

1/1 [==============================] - 0s 28ms/step


100%|███████████████████████████████████████████████████████████████████████████████▌| 199/200 [00:23<00:00,  9.16it/s]

1/1 [==============================] - 0s 28ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:23<00:00,  8.59it/s]


In [4]:
def read_image(path):
    x = cv.imread(path, cv.IMREAD_COLOR)
#     x = x/255.0
#     x = x.astype(np.float32)
    return x

folder_path= os.path.join('Binarized/*')

for subdir in glob.glob(folder_path):
    if __name__ == "__main__": 
        
        """ Load the dataset """
        test_x = sorted(glob.glob(os.path.join(subdir, "*.png")))
        
        i=0
        for z in tqdm(zip(test_x), total=len(test_x)):
            ori_x = read_image(z[i])
            i=i+1
        vol = np.stack([ori_x], axis=0)
        print(vol.shape)

        



AttributeError: 'function' object has no attribute 'glob'